In [1]:
import pandas as pd
import numpy as np


def limpiar_dataset(df):
    
    # Dropear columnas
    
    rbnb_clean = df.drop(

    [
    # 'host_since',
    'host_neighbourhood',
    'host_verifications',
    #'neighbourhood_cleansed',
    #'property_type',
    #'room_type',
    #'accommodates,
    #'bathrooms_text',
    #'bedrooms',
    #'beds',
    #'amenities',
    #'availability_365',   
    #'number_of_reviews',
    #'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'license',
    #'calculated_host_listings_count_private_rooms',
        
    'host_location',
    'host_acceptance_rate',

    'id',
    'listing_url',
    'scrape_id',
    'last_scraped',
    'name',
    'description',
    'neighborhood_overview',
    'picture_url',
    'host_id',
    'host_url',
    'host_name',
    'host_about',
    'host_response_time',
    'host_response_rate',
    'host_is_superhost',
    'host_thumbnail_url',
    'host_picture_url',
    'host_listings_count',
    'host_total_listings_count',
    'host_has_profile_pic',
    'host_identity_verified',
    'neighbourhood',
    'neighbourhood_group_cleansed',
    'latitude',
    'longitude',
    'bathrooms',
    'minimum_nights',
    'maximum_nights',
    'minimum_minimum_nights',
    'maximum_minimum_nights',
    'minimum_maximum_nights',
    'maximum_maximum_nights',
    'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm',
    'calendar_updated',
    'has_availability',
    'availability_30',
    'availability_60',
    'availability_90',
    'calendar_last_scraped',
    'number_of_reviews_ltm',
    'number_of_reviews_l30d',
    'first_review',
    'last_review',
    'instant_bookable',
    'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_shared_rooms',
    'reviews_per_month',
        
    ],
    axis=1)
    
    # Modifico la 'host_since' para sacarle los días que llevan alquilando.

    rbnb_clean.host_since = rbnb_clean.host_since.apply(pd.to_datetime)

    rbnb_clean.host_since = rbnb_clean.host_since.apply(lambda x: pd.Timestamp.now() - x)

    rbnb_clean.host_since = rbnb_clean.host_since.apply(lambda x: int(x / np.timedelta64(1, 'D')))
    
    
    
    # Modifico la columna 'bathrooms_text' para extraer de la string el número de baños

    def extract_bathrooms(x):
        try:
            return re.findall('\d+', x)[0]
        except:
            return 0.5
    rbnb_clean.bathrooms_text = rbnb_clean.bathrooms_text.apply(extract_bathrooms)
    rbnb_clean.bathrooms_text = rbnb_clean.bathrooms_text.astype(float)
    
    
    # Modifico la columna 'amenities' para quitarle caracteres.

    rbnb_clean.amenities = rbnb_clean.amenities.apply(lambda x: x.replace('"', '').replace('[', '').replace(']', ''))
    rbnb_clean.amenities = rbnb_clean.amenities.apply(lambda x: x.count(','))
    
    # Empiezo a rellenar valores nulos

    rbnb_clean.bedrooms = rbnb_clean.bedrooms.fillna(1)

    rbnb_clean.beds = rbnb_clean.beds.fillna(1)

    rbnb_clean.review_scores_rating = rbnb_clean.review_scores_rating.fillna(rbnb_clean.review_scores_rating.median())
    
    
    # Reduzco los valores unicos de columnas categoricas

    top_neighbourhood = rbnb_clean.neighbourhood_cleansed.value_counts().head(12).index
    rbnb_clean.neighbourhood_cleansed[~rbnb_clean.neighbourhood_cleansed.isin(top_neighbourhood)] = 'OTHER'

    top_property_type = rbnb_clean.property_type.value_counts().head(12).index
    rbnb_clean.property_type[~rbnb_clean.property_type.isin(top_property_type)] = 'OTHER'
    
    
    return rbnb_clean

In [2]:
'''from sklearn.preprocessing import StandardScaler

def standarizar(df):

    data_num=pd.DataFrame(StandardScaler().fit_transform(df._get_numeric_data()),  # standardize numeric columns
                          columns=df._get_numeric_data().columns)

    data_obj=df.select_dtypes(include='object')  # get categoric columns


    data=pd.concat([data_num, data_obj], axis=1)   # concatenate both dataframes

    return data'''

"from sklearn.preprocessing import StandardScaler\n\ndef standarizar(df):\n\n    data_num=pd.DataFrame(StandardScaler().fit_transform(df._get_numeric_data()),  # standardize numeric columns\n                          columns=df._get_numeric_data().columns)\n\n    data_obj=df.select_dtypes(include='object')  # get categoric columns\n\n\n    data=pd.concat([data_num, data_obj], axis=1)   # concatenate both dataframes\n\n    return data"

In [3]:
from sklearn.preprocessing import StandardScaler

def standarizar(df):
    
    scaler=StandardScaler()

    df[['host_since', 'availability_365', 'number_of_reviews']]=scaler.fit_transform(df[['host_since', 'availability_365', 'number_of_reviews']])
    
    return df

In [4]:
def one_hot_encoding(df):

    rbnb_clean = pd.get_dummies(df, columns=df.select_dtypes(include=object).columns, drop_first=True)
    
    return rbnb_clean

In [5]:
from sklearn.model_selection import train_test_split

def separar_splitear_xy(df):
    
    # Separo la X de la y

    X=df.drop('price', axis=1)

    y=df.price

    # Hago un split del 80% para entrenar y 20% para testear


    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, train_size=0.8,
                                                        random_state=42)

    return X_train, X_test, y_train, y_test

In [6]:
from sklearn.model_selection import GridSearchCV

def grid(modelo, param, cv=5):
    
    grid=GridSearchCV(modelo, param, cv=cv,
                      return_train_score=True, n_jobs=-1)
    
    grid.fit(X_train, y_train)
    
    print('Acierto test: {:.2f}'.format(grid.score(X_test, y_test)))
    print('Acierto train: {:.2f}'.format(grid.score(X_train, y_train)))
    print('Mejores parametros: {}'.format(grid.best_params_))
    print('Mejor acierto cv: {:.2f}'.format(grid.best_score_))
    
    return grid.best_estimator_.fit(X_train, y_train)